# Aplicando el problema de la mochila a optimización de carteras
El problema de la mochila se puede aplicar a optimización de carteras de inversión, definiéndolo de forma similar al problema de la mochila:
- Existe un conjunto de posibles inversiones a realizar. Cada una de ellas tiene un retorno esperado ($r_x$) y una cantidad a invertir (coste, $c_x$).
- Hay una restricción: la cantidad máxima a invertir, $C$.

En ese sentido, el problema es análogo al definido hasta ahora. No es muy interesante ni práctico en escenarios reales.

Incorporaremos dos elementos adicionales al problema:

- **Minimización del riesgo**: Cada inversión posible en un activo financiero tiene un riesgo asociado. Este se puede descomponer en dos aspectos:
  - Riesgo intrínseco: el riesgo del propio activo. Por ejemplo, su volatilidad o varianza.
  - Riesgo compuesto: el riesgo asociado a la inversión conjunta en dos activos financieros. Por ejemplo, si ambos activos están altamente correlacionados (invertir en dos empresas del mismo sector), el riesgo global de la inversión aumenta. Sin embargo, si se invierte en activos inversamente correlacionados, el riesgo global de la inversión se reduce (si un activo baja, el otro subirá para compensarlo).
- **Inversiones no binarias**: Hasta ahora hemos trabajado con objetos que podían inclurise o no en la mochila. Este modelo es válido por ejemplo para inversiones en las que se adquiere un activo completo (comprar toda la empresa, o todo lo que está disponible para la venta) o se asume un gasto para obtener un retorno (abrir nuevas sucursales de una empresa). No obstante, para ciertos activos financieros la posible inversión no es binaria (invertir o no invertir) sino que es discreta (comprar un número concreto de acciones). Incorporaremos esta opción al problema de la mochila, reformulándolo como un QUBO.

## Formulación matemática

### Retorno esperado

El retorno se expresa comunmente de dos formas distintas:

#### Retorno absoluto por unidad

Si el retorno esperado por unidad del activo $i$ es $r_i$ (en unidades monetarias):

$$
R_i = \sum_i r_i x_i
$$

#### Retorno relativo (ROI)

Si el retorno se expresa como un porcentaje $\rho_i$ (ROI), el retorno monetario esperado por unidad es $\rho_i p_i$:

$$
R_i = \sum_i (\rho_i p_i) x_i
$$

En ambos casos, el término de retorno se introduce en el QUBO con signo negativo. **Nosotros utilizaremos la alternativa de Retorno Relativo**


### Formulación matemática de la minimización del riesgo
El parámetro que mide el riesgo por la interacción entre dos inversiones se expresa como una relación (covarianza) entre dos variables (el retorno de dos inversiones distintas), y mide como cambia una respecto a la otra (qué pasa al retorno de una inversión cuando la otra cambia en una dirección concreta). Denominamos a esta covarianza $\sigma_{i j}$:

- Si cuando la invesión $i$ va mejor de lo esperado, la inversion $j$ también va mejor de lo esperado, $\sigma_{i j} > 0$
- Si cuando la invesión $i$ va peor de lo esperado, la inversion $j$  va mejor de lo esperado, $\sigma_{i j} < 0$

Por otro lado, el riesgo intrínseco mide la varianza de un único activo:

- $\sigma_{i i}$ sería la varianza propia del activo, una medición de su volatilidad.

Usando ambos parámetros, podemos formular un término de riesgo que queremos minimizar. 

$$ \sum_{i=0}^{n}\sum_{j=0}^{n} \sigma_{ij} c_i c_j x_i x_j $$

### La formulación completa
Incorporando el retorno y el riesgo a la formulación estandar del problema de la mochila (en este caso, con restricción de igualdad, aunque es indiferente), usando un nuevo parámetro $\beta$ para controlar la aversión al riesgo, obtenemos:

$$\underbrace{-\sum_{i=0}^{n} R_i }_{\text{Maximizar Retorno}} + \underbrace{\alpha \left(\sum_{i=0}^{n} c_i x_i - C\right)^2}_{\text{Restricción de Presupuesto}} + \underbrace{\beta \sum_{i=0}^{n}\sum_{j=0}^{n} \sigma_{ij} c_i c_j x_i x_j}_{\text{Minimizar Riesgo}}$$

Como podemos observar, la formulación es idéntica a la del problema de la mochila, pero incorpora un término adicional para el riesgo. Por tanto, con respecto a nuestra solución anterior para el problema de la mochila, solo debemos incorporar estos términos adicionales en función de $\sigma$:

- Términos lineales: $\beta \sigma_{i i} c_i^2 x_i$
- Términos cuadráticos: $\beta \sigma_{i j} c_i c_j x_i x_j$

**La estimación del parámetro $\beta$** se calcula como la proporción entre el ROI medio y la varianza media de los activos. Este procedimiento nivela la importancia relativa de ambos términos. De este modo, se garantiza que el retorno y el riesgo tengan un peso equilibrado en la búsqueda de la solución, consiguiendo una cartera estable

Añadiendo los mismos a nuestra formulación como QUBO para el problema de la mochila, obtendríamos la solución al problema de optimización de carteras. Podemos añadir estos términos tanto a la formulación con variables de holgura (restricción de desigualdad) como a la formulación estándar (restricción de igualdad).


## Clase para abstraer un problema de optimización de carteras
A continuación desarrollamos una clase que encapsula el problema de optimización de carteras. Almacena todos los activos financieros y sus características (coste, retorno, riesgo) y las características generales del problema (inversión máxima)

La clase distingue dos conceptos fundamentales:
- Activos financieros: en los que se puede invertir, con coste, retorno y riesgo
- Variable de decisión: una variable para el QUBO, que corresponde a un activo financiero pero permite incorporar variables de holgura y permitirá capturar inversiones múltiples (no binarias).

Además, la clase encapsula la generación de las variables a partir de los activos. Inicialmente, mediante dos métodos separados, permite generar las variables simples (que corresponden 1:1 con activos financieros) y variables de holgura. Más adelante extenderemos la clase para incorporar otras variables.

In [ ]:
import math
from tabulate import tabulate
import copy
import numpy as np
import pandas as pd
import random
import string
import statistics


# Función auxiliar para inyectar métodos a una clase
# Nos permitirá añadir métodos a la clase más adelante en el notebook
def add_method(cls):
    def decorator(func):
        setattr(cls, func.__name__, func)
        return func
    return decorator



class FinancialAsset:
    """
    Representa un activo financiero.
    """
    def __init__(self, name, price, roi, variance=0):
        self.name = name
        self.price = price       # Coste por unidad (c)
        self.roi = roi           # Retorno esperado (r) como ratio (ej. 1.05)
        self.variance = variance # Riesgo intrínseco (sigma^2)

    def __str__(self):
        return self.name

class DecisionVariable:
    """
    Representa una variable para nuestro QUBO. Pueden ser un mapeo directo de FinancialAsset
    o otro tipo de variables (holgura, descomposición binaria...)
    """
    def __init__(self, parent_name, quantity, cost, value, is_slack=False, other_vars = None):
        self.parent_name = parent_name # Nombre del activo "padre" de este
        self.quantity = quantity # Cantidad de activos (si representa varias unidades del mismo activo)
        self.cost = cost # Coste de esta variable (precio del activo por número de activos)
        self.value = value # Valor esperado total de esta variable (ROI * coste)
        self.is_slack = is_slack # Booleano para identificar variables de holgura
        self._full_name = self.parent_name + "_" + str(self.quantity)

        if other_vars is not None:
            while self._full_name in [v._full_name for v in other_vars]:
                self._full_name += "*"

    def name(self):
        return self._full_name

class PortfolioProblem:
    """
    Clase constructora del problema de optimización.
    """
    def __init__(self, max_budget):
        self.max_budget = max_budget
        self.assets = {} # indexados por nombre: name -> FinancialAsset
        self.covariances = {} # indexados por pares de nombres: (name, name) -> float
        self.variables = [] # Lista de DecisionVariable
        self.variables_index = None

    def copy(self):
        return copy.deepcopy(self)
    
    
    def add_asset(self, name, price, variance=0, roi_ratio=None, expected_value=None):
        """
        Registra un activo. Puede tomar un ROI o un valor esperado (y almacena el ROI calculado)
        """
        final_roi = roi_ratio
        if final_roi is None:
            if expected_value is None:
                final_roi = 0
            else:
                final_roi = expected_value / price

        self.assets[name] = FinancialAsset(name, price, final_roi, variance)

    def add_covariance(self, asset1_name, asset2_name, value):
        """Registra la covarianza entre dos activos."""
        key = tuple(sorted((asset1_name, asset2_name)))
        self.covariances[key] = value

    def get_asset_risk(self, name1, name2=None):
        """Devuelve la covarianza (o varianza si es el mismo activo)."""
        if name2 == None or name1 == name2:
            return self.assets[name1].variance
        key = tuple(sorted((name1, name2)))
        return self.covariances.get(key, 0.0)

    def get_variable_by_full_name(self, full_name):
        if self.variables_index == None or full_name not in self.variables_index:
            self.variables_index = {}
            for v in self.variables:
                self.variables_index[v._full_name]=v
        return self.variables_index[full_name]

    def clear_variables(self):
        """Limpia la lista de variables."""
        self.variables = []

    def create_variables_simple(self):
        """
        Crea las variables para un problema mochila básico: Una variable por activo.
        """
        for asset in self.assets.values():
            if asset.price <= self.max_budget:
                val = asset.price * asset.roi
                self.variables.append(DecisionVariable(asset.name, 1, asset.price, val, other_vars=self.variables))
    
    def create_slack_variables(self):
        """
        Estrategia: Variables de holgura (Slack).
        """
        
        # Tenemos que sumar 1 ya que si el número máximo es justo una potencia de dos
        # no cubriríamos todo (log2(8)=3, 2^0 + 2^1 +2^2 = 7)
        k = int(math.floor((math.log2(self.max_budget))+1))
        
        for i in range(k):
            val = 2**i                
            self.variables.append(DecisionVariable("SLACK", int(val), float(val), 0.0, is_slack=True, other_vars=self.variables))

    def print_problem_definition(self, display_covariances = False, display_variables=False):
        """
        Imprime los activos disponibles, covarianzas y estado de variables generadas.
        """
        print("\n=== Definición del Problema ===")
        
        print(f"Presupuesto Máximo (C): {self.max_budget}")

        # Tabla de Activos
        print("\n--- Activos Disponibles ---")
        table_data = []
        for a in self.assets.values():
            table_data.append([a.name, a.price, f"{a.roi:.4f}", a.variance])
        
        print(tabulate(table_data, headers=["Activo", "Coste Unit.", "ROI (Ratio)", "Riesgo intrínseco (Varianza)"]))
            
        # Tabla de Covarianzas
        if display_covariances and self.covariances:
            print("\n--- Riesgos Conjuntos (Covarianzas) ---")
            cov_data = [[k[0], k[1], v] for k, v in self.covariances.items()]
            print(tabulate(cov_data, headers=["Activo A", "Activo B", "Valor"]))

        # Resumen de Variables
        print("\n--- Estado del Modelo ---")
        print(f"Variables de decisión generadas: {len(self.variables)}")
        slack_count = sum(1 for v in self.variables if v.is_slack)
        if slack_count > 0:
            print(f"(Incluye {slack_count} variables de holgura)")
        if(display_variables):
            print("\n--- Variables ---")
            table_data = []
            for a in self.variables:
                table_data.append([a.name(), a.quantity, f"{a.cost:.4f}", f"{a.value:.4f}"])
            
            print(tabulate(table_data, headers=["Activo", "Unidades", "Coste", "Valor"]))  


    def get_solution_table(self, solution_vector):
        """
        Para un vector de solución dado, entrega un diccionario con todas las acciones, la
        cantidad a comprar, el coste total y el retorno esperado total:

        name -> {'quantity': 0, 'cost': 0.0, 'value': 0.0}
        """
        summary = {}
        for name, selected in solution_vector.items():
            if selected > 0:
                var = self.get_variable_by_full_name(name)
                
                if not var.is_slack:
                    if var.parent_name not in summary:
                        summary[var.parent_name] = {'quantity': 0, 'cost': 0.0, 'value': 0.0}
                    
                    summary[var.parent_name]['quantity'] += var.quantity
                    summary[var.parent_name]['cost'] += var.cost
                    summary[var.parent_name]['value'] += var.value
        return summary
        
    def print_solution(self, solution_vector, solution_info = None, noprint = False):
        """
        Imprime la solución
        """
        if not noprint:
            print("\n=== Solución ===")
        if len(solution_vector) != len(self.variables):
            if not noprint:
                print("Error: Vector solución de tamaño incorrecto.")
                print("Solucion: ", solution_vector)
                print("Variables: ", [v._full_name for v in self.variables])
            return

        summary = {}
        total_inv = 0
        total_ret = 0
        slack_inv = 0

        for name, selected in solution_vector.items():
            if selected > 0:
                var = self.get_variable_by_full_name(name)
                
                if var.is_slack:
                    slack_inv += var.cost
                else:
                    if var.parent_name not in summary:
                        summary[var.parent_name] = {'u': 0, 'c': 0.0, 'v': 0.0}
                    
                    summary[var.parent_name]['u'] += var.quantity
                    summary[var.parent_name]['c'] += var.cost
                    summary[var.parent_name]['v'] += var.value
                    
                    total_inv += var.cost
                    total_ret += var.value

        data = []
        for name, info in summary.items():
            pct = (info['c'] / self.max_budget) * 100
            data.append([name, info['u'], f"{info['c']:.2f}", f"{pct:.1f}%", f"{info['v']:.2f}"])
            
        if slack_inv > 0:
            data.append(["[HOLGURA]", "-", f"{slack_inv:.2f}", f"{(slack_inv/self.max_budget)*100:.1f}%", "0.0"])

        if not noprint:
            print(tabulate(data, headers=["Activo", "Unidades", "Inversión", "% Total", "Retorno"]))
            print(f"Total Invertido en Activos: {total_inv:.2f}")
            print(f"Total presupuesto: {self.max_budget:.2f}")
            print(f"Holgura (no invertido): {slack_inv:.2f}")
            print(f"Total invertido + holgura: {total_inv + slack_inv:.2f}", f"( {100*(total_inv + slack_inv)/self.max_budget:.2f} %)")
            print(f"Total retorno: {total_ret:.2f}")

        risk = 0
        active_assets = list(summary.keys())
        for n1 in active_assets:
            for n2 in active_assets:
                c1 = summary[n1]['c']
                c2 = summary[n2]['c']
                risk += c1 * c2 * self.get_asset_risk(n1, n2)
        if not noprint:
            print(f"Riesgo Total (Varianza): {risk:.4f}")

        if solution_info is not None:
            solution_info.num_variables = len(self.variables)
            solution_info.retorno = total_ret
            solution_info.inversion = total_inv
            solution_info.riesgo = risk

class PortfolioSolution:
    """
    Clase para guardar los parámetros de una solución.
    """
    def __init__(self, solucion = None, num_variables = None, retorno = None, riesgo = None, inversion = None, alpha = None, beta = None, run_time = None):
        self.num_variables = num_variables
        self.run_time = run_time
        self.retorno = retorno
        self.riesgo = riesgo
        self.inversion = inversion
        self.alpha = alpha
        self.beta = beta
        self.solucion = solucion

In [ ]:
## Código para generar problemas aleatorios y de ejemplo
from dimod import ExactSolver

def create_random_problem(max_investment, num_assets):    
    problema = PortfolioProblem(max_investment)
    
    for i in range (num_assets):
        name = ''.join(random.choice(string.ascii_uppercase) for _ in range(4))
        price = round(random.lognormvariate(3.5, 1.0), 2) # lognormvariate porque queremos precios positivos, con sigma alto para tener bastante dispersión
        roi_ratio = round(random.normalvariate(0.08, 0.20), 4) # Media 0.08 (8% histórico aprox) y desviación 0.20 (20% volatilidad estándar)
        roi_ratio = round(random.normalvariate(0.08, 0.20)**2, 4)
        variance = round(random.gammavariate(2, 0.04), 4) # Media de 0.08 con cola a la derecha. Valores muy concentrados pero con algún activo muy volatil
        problema.add_asset(name, price, variance, roi_ratio)
    
    
    #Para estimar la covarianza se utiliza la correlación de Pearson con un coeficiente aleatorio
    for a1 in problema.assets.values():
        std_1 = math.sqrt(a1.variance) #volatilidad real, la raiz de la varianza
        for a2 in problema.assets.values():
            if a1.name >= a2.name: # Solo añadimos una diagonal de la matriz
                continue
            std_2 = math.sqrt(a1.variance) #volatilidad real, la raiz de la varianza
            rho = random.uniform(-0.2, 0.8) # La correlación es aleatoria
            cov = rho * std_1 * std_2 # Fórmula de la covarianza en función de la volatilidad y la correlación
            problema.add_covariance(a1.name, a2.name, cov)
    return problema
    
problema_random = create_random_problem(max_investment = 1000000, num_assets = 7)

problema_random.print_problem_definition()

problema_estatico = PortfolioProblem(200)

problema_estatico.add_asset("1", 100, 0.15, 0.1)
problema_estatico.add_asset("2", 200, 0.10, 0.05)
problema_estatico.add_asset("3", 50, 0.02, -0.05)
problema_estatico.add_asset("4", 50, 0.15, -0.25)
problema_estatico.add_asset("5", 75, 0.05, 0.15)
problema_estatico.add_asset("6", 75, 0.06, 0.13)

for a in problema_estatico.assets.values():
    for b in problema_estatico.assets.values():
        if (a.name == "1" and b.name =="5"):
            std_1 = math.sqrt(a.variance)
            std_2 = math.sqrt(b.variance)
            rho = 0.2
            cov = rho * std_1 * std_2
            problema_estatico.add_covariance(a.name, b.name, cov)
        elif (a.name == "5" and b.name =="1"):
            std_1 = math.sqrt(a.variance)
            std_2 = math.sqrt(b.variance)
            rho = 0.2
            cov = rho * std_1 * std_2
            problema_estatico.add_covariance(a.name, b.name, cov)
        elif (a.name == "5" and b.name =="6"):
            std_1 = math.sqrt(a.variance)
            std_2 = math.sqrt(b.variance)
            rho = -0.2
            cov = rho * std_1 * std_2
            problema_estatico.add_covariance(a.name, b.name, cov)
        elif (a.name == "6" and b.name =="5"):
            std_1 = math.sqrt(a.variance)
            std_2 = math.sqrt(b.variance)
            rho = -0.2
            cov = rho * std_1 * std_2
            problema_estatico.add_covariance(a.name, b.name, cov)
        else:
            problema_estatico.add_covariance(a.name, b.name, 0)

problema_estatico.print_problem_definition()



In [ ]:
# Resolvemos un QUBO básico para el problema anterior
# Usamos variables binarias (invertir o no) y NO incorporamos variables de holgura ni riesgo

def solve_qubo_basico_vars(problema, print_solution = True):
    ## QUBO básico tipo mochila. Sin riesgo ni multi-inversiones
    qubo_basico = {}
    
    # Calculamos alpha buscando el valor máximo entre los objetos
    alpha = max([o.value for o in problema.variables]) * 1.1
    
    for o_i in problema.variables:
        # Término lineal
        qubo_basico[(o_i.name(), o_i.name())] = (
            - o_i.value
            + alpha * (o_i.cost**2)
            - 2 * alpha * problema.max_budget * o_i.cost
        )
    
        for o_j in problema.variables:
            if o_i.name() == o_j.name():
              continue
            # Término cuadrático
            qubo_basico[(o_i.name(), o_j.name())] = alpha * o_i.cost * o_j.cost
    exact_solver = ExactSolver()
    soluciones_qubo_basico = exact_solver.sample_qubo(qubo_basico)
    solucion_qubo_basico = soluciones_qubo_basico.first.sample
    
    solution_info = PortfolioSolution()
    solution_info.solucion = solucion_qubo_basico
    problema.print_solution(solucion_qubo_basico, solution_info = solution_info, noprint = not print_solution)
    solution_info.alpha = alpha
    solution_info.num_variables = len(problema.variables)
    return solution_info

In [ ]:
problema_estatico_2 = problema_estatico.copy()
problema_estatico_2.create_variables_simple()
problema_estatico_2.print_problem_definition()
sol_info = solve_qubo_basico_vars(problema_estatico_2)

In [ ]:
# Resolvemos un QUBO básico para el problema anterior
# Usamos variables binarias (invertir o no) y SÍ incorporamos variables de holgura

problema_estatico_3 = problema_estatico.copy()
problema_estatico_3.create_variables_simple()
problema_estatico_3.create_slack_variables()
problema_estatico_3.print_problem_definition()
sol_info = solve_qubo_basico_vars(problema_estatico_3)

## Inversiones no binarias: incorporando descomposición binaria de las variables

En escenarios reales de optimización de carteras, la decisión no suele ser binaria, sino discreta: se decide cuántas unidades de cada activo adquirir.

Denotamos por:

$$
q_i \in \{0,1,2,\dots,Q_i^{\max}\}
$$

el número de unidades del activo $i$ que se compran.

### Descomposición binaria de las cantidades

Dado que los modelos QUBO solo admiten variables binarias, representamos cada cantidad $q_i$ mediante una descomposición binaria:

$$
q_i = \sum_{k=0}^{K_i} 2^k\,x_{i,k},
\quad x_{i,k}\in\{0,1\}
$$

donde:

$$
K_i = \left\lfloor \log_2(Q_i^{\max}) \right\rfloor,
\qquad
Q_i^{\max} = \left\lfloor \frac{C}{p_i} \right\rfloor
$$

siendo:
- $p_i$ el precio por unidad del activo $i$
- $C$ el presupuesto máximo disponible

Cada variable binaria $x_{i,k}$ representa la compra de un “paquete” de $2^k$ unidades del activo $i$.

### Coste total de la cartera

El coste total de la inversión es:

$$
\text{Coste} = \sum_i p_i q_i
= \sum_i \sum_k (2^k p_i)\,x_{i,k}
$$

Por tanto, cada variable $x_{i,k}$ tiene asociado un coste:

$$
c_{i,k} = 2^k p_i
$$

### Retorno esperado

Como hemos visto, el retorno se puede formular de dos formas. Veamos como se adapta cada una a una inversión no binaria:

#### Retorno absoluto por unidad

Si el retorno esperado por unidad del activo $i$ es $r_i$ (en unidades monetarias):

$$
R_{i,k} = \sum_i r_i q_i
= \sum_i \sum_k (2^k r_i)\,x_{i,k}
$$

#### Retorno relativo (ROI)

Si el retorno se expresa como un porcentaje $\rho_i$ (ROI), el retorno monetario esperado por unidad es $\rho_i p_i$:

$$
R_{i,k} = \sum_i (\rho_i p_i) q_i
= \sum_i \sum_k (2^k \rho_i p_i)\,x_{i,k}
$$

En ambos casos, el término de retorno se introduce en el QUBO con signo negativo:

$$
-\sum_{i,k} R_{i,k}\,x_{i,k}
$$


### Riesgo de la cartera con inversiones no binarias

El riesgo de una cartera se modela mediante la matriz de covarianzas $\sigma_{ij}$.

Para cantidades no binarias, el riesgo total es:

$$ \text{Riesgo} = \sum_{i=1}^n \sum_{j=1}^n \sigma_{ij}\,p_i q_i p_j q_j $$

Sustituyendo la descomposición binaria:

$$
p_i q_i p_j q_j
= \left(\sum_k 2^k p_i x_{i,k}\right)\left(\sum_l 2^l p_j x_{j,l}\right)
= \sum_k \sum_l 2^{k+l} p_i x_{i,k} p_j x_{j,l}
$$


Por tanto:

$$
\text{Riesgo}
= \sum_{i,j}\sum_{k,l}\sigma_{ij}\,2^{k+l}\,p_i x_{i,k} p_j x_{j,l}
$$

Teniendo en cuenta que $c_{i,k} = 2^k \, p_i$:

$$
\text{Riesgo}
= \sum_{i,j}\sum_{k,l}\sigma_{ij}\, c_{i,k} c_{j,l} x_{i,k} x_{j,l}
$$



Este término penaliza tanto el riesgo intrínseco ($i=j$) como el riesgo conjunto entre activos correlacionados ($i\neq j$).

Mantenemos el parámetro $\beta$ para controlar la aversión al riesgo.

## Función de energía QUBO final
La función de energía a minimizar queda:

$$
    E(x) =
    \underbrace{-\sum_{i,k} R_{i,k}x_{i,k}}_{\text{Retorno}}
    + \underbrace{\beta\sum_{i,j}\sum_{k,l}\sigma_{ij}c_{i,k} c_{j,l}x_{i,k} x_{j,l}}_{\text{Riesgo}}
    + \underbrace{\alpha\left(\sum_{i,k} c_{i,k}x_{i,k} - C\right)^2}_{\text{Condición de igualdad}}
$$


Se desarrolla el cuadrado y se agrupan los términos en lineales y cuadráticos

$$
\begin{aligned}
    E(x) = &
    \sum_{\substack{i,k}}( -R_{i,k} + \beta \, \sigma_{i,i} \, c_{i,k}^2+ \alpha \, c_{i,k}^2 - 2 \alpha \, C \, c_{i,k})x_{i,k} \\
    & + \sum_{\substack{i,j \\ i \neq j}} \sum_{\substack{k,l}}(\beta \, \sigma_{i,j} \, c_{i,k} \, c_{j,l} + \alpha \, c_{i,k} \, c_{j,l})x_{i,k}x_{j,l}
\end{aligned}
$$

Esta formulación permite resolver el problema de optimización de carteras con inversiones no binarias dentro del marco QUBO.



In [ ]:
@add_method(PortfolioProblem)
def get_interaction_risk(self, var_i, var_j):
    """Calcula el riesgo cuadrático entre dos variables, teniendo en cuenta que pueden estar paquetizadas"""

    # Las variables slack no incrementan el riesgo, así que devolvemos 0
    if var_i.is_slack or var_j.is_slack:
        return 0.0

    # El riesgo aumenta con la cantidad de dinero invertido en cada una de las variables (ya que 
    # la covarianza está expresada para cantidad 1 de cada variable)
    cov = self.get_asset_risk(var_i.parent_name, var_j.parent_name)
    return var_i.cost * var_j.cost * cov

@add_method(PortfolioProblem)
def create_variables_binary_simple(self):
    """
    Crea las variables usando descomposición binaria. Se permite un número arbitrario de 
    unidades de cada activo.
    """
    for asset in self.assets.values():
        # Calculamos cuantas acciones puedo tener
        max_units = math.floor(self.max_budget / asset.price)

        # Si no me entra ni siquiera una acción, no añado variables
        if max_units < 1:
            continue
        # Tenemos que sumar 1 ya que si el número máximo es justo una potencia de dos
        # no cubriríamos todo (log2(8)=3, 2^0 + 2^1 +2^2 = 7)
        k = int(math.floor((math.log2(max_units))+1))

        for i in range(k):
            count = 2**i
            cost = count * asset.price
            val = cost * asset.roi
            self.variables.append(DecisionVariable(asset.name, count, cost, val, other_vars=self.variables))



def solve_qubo_basico_risk(problema, alpha=None, beta=None, print_solution = True):
    ## QUBO básico tipo mochila. Sin riesgo ni multi-inversiones
    qubo_basico = {}
    
    # Calculamos alpha buscando el valor máximo entre los objetos
    if alpha == None:
        alpha = max([o.value for o in problema.variables]) * 1.1
    if print_solution:
        print ("Alpha: ", alpha)

    avg_roi = statistics.mean([a.value 
                               for a in problema.variables
                               if not a.is_slack])
    avg_cov = statistics.mean([a.cost**2 * problema.get_asset_risk(a.parent_name) 
                               for a in problema.variables
                               if not a.is_slack])
    if beta == None:    
        beta = abs(avg_roi/avg_cov)#Evitamos betas negativos
    if print_solution:
        print ("Beta: ", beta, avg_roi, avg_cov)
    
    for o_i in problema.variables:
        varianza = 0
        if not o_i.is_slack:
            varianza = problema.get_asset_risk(o_i.parent_name)
        # Término lineal
        qubo_basico[(o_i.name(), o_i.name())] = (
            - o_i.value
            + alpha * (o_i.cost**2)
            - 2 * alpha * problema.max_budget * o_i.cost
            + beta * varianza * o_i.cost**2 
        )
    
        for o_j in problema.variables:
            if o_i.name() == o_j.name():
              continue
            covarianza = 0
            if not o_i.is_slack and not o_j.is_slack:
                covarianza = problema.get_asset_risk(o_i.parent_name, o_j.parent_name)  
            # Término cuadrático
            qubo_basico[(o_i.name(), o_j.name())] = (  
                alpha * o_i.cost * o_j.cost
                + beta * o_i.cost * o_j.cost * covarianza
             )
    exact_solver = ExactSolver()
    soluciones_qubo_basico = exact_solver.sample_qubo(qubo_basico)
    solucion_qubo_basico = soluciones_qubo_basico.first.sample
    solution_info = PortfolioSolution()
    solution_info.solucion = solucion_qubo_basico
    problema.print_solution(solucion_qubo_basico, solution_info = solution_info, noprint = not print_solution)
    solution_info.alpha = alpha
    solution_info.beta = beta
    solution_info.num_variables = len(problema.variables)
    return solution_info


In [ ]:
## Código para ejecutar un QUBO con variables no binarias, usando la clase y el nuevo riesgo de interacción

problema_descomposicion = problema_estatico.copy()
problema_descomposicion.create_variables_binary_simple()
problema_descomposicion.create_slack_variables()

problema_descomposicion.print_problem_definition(False, False)

sol_info = solve_qubo_basico_risk(problema_descomposicion)



## Optimizando el uso de variables

Con la formulación actual, para cada posible acción necesitamos $log_2{C/c_i}$ variables, a las que hay que añadir las variables de holgura. Esto hace que la complejidad del problema crezca rápidamente.

Existen diversas técnicas para optimizar el uso de variables:
- No usar variables de holgura (ya están implementadas ambas opciones)
- Limitar que un único activo ocupe más de un determinado porcentaje de la cartera
- Crear paquetes de acciones (reducir la granularidad)

Veamos como se pueden implementar cada una de estas optimizaciones.

### No usar variables de holgura
Puesto que podemos hacer inversiones de cantidades arbitrarias de cada activo, si no incluímos variables de holgura, el modelo intentará maximizar las inversiones para usar todo el presupuesto. Esto es una buena aproximación, ya que en general el inversor querrá invertir todo su presupuesto, y nos permite evitar el uso de variables de holgura, usando simplemente una restricción de igualdad.

#### Restricción de igualdad

Si se desea invertir exactamente el presupuesto disponible:

$$
\alpha\left(\sum_{i,k} c_{i,k}x_{i,k} - C\right)^2
$$

#### Restricción de desigualdad (con holgura)

Si se permite no invertir todo el presupuesto:

$$
\sum_{i,k} c_{i,k}x_{i,k} + S = C,
\qquad
S = \sum_{m=0}^{M}2^m y_m
$$

donde $y_m \in \{0,1\}$ son variables binarias auxiliares sin retorno asociado.

## Código para ejecutar sin variables de holgura y comparar los resultados


In [ ]:
## Versión sin variables de holgura
problema_optimización_noHolgura = problema_estatico.copy()
problema_optimización_noHolgura.create_variables_binary_simple()
problema_optimización_noHolgura.print_problem_definition(False, False)

sol_info = solve_qubo_basico_risk(problema_optimización_noHolgura)

### Crear paquetes de acciones
Podemos representar un activo financiero como un múltiplo de la acción individual, y calcular proporcionalmente su coste, retorno y riesgo. De este modo, al ser el coste mayor respecto al coste total, necesitaremos menos variables para poder llenar la mochila con este activo. Esta opción se puede implementar con paquetes arbitrariamente grandes, si bien reduce la capacidad del modelo de ajustar de forma precisa (es decir, no encontraremos el mínimo local real).

En caso de usar un tamaño mínimo del paquete, podemos aplicarlo también a las slack variables, que deben ser múltiplos de este paquete, con una variable adicional para cubrir el último hueco no usable.

In [ ]:
@add_method(PortfolioProblem)
def create_variables_binary_packaged(self, min_package_pct=0.05):
    """
    Crea las variables usando descomposición binaria. Se permiten varias unidades de cada activo,
    pero se permite solo un número mínimo de unidades y múltiplos de ese número.
    Así, en caso de acciones muy baratas, reducimos el número de variables.
    El número mínimo de unidades se calcula en función del presupuesto total (tamaño de la cartera)
    """
    min_money_package = self.max_budget * min_package_pct
    
    for asset in self.assets.values():
        # Cuantas acciones son un paquete?
        if asset.price >= min_money_package:
            base_units = 1
        else:
            base_units = round(min_money_package / asset.price)

        # Cuantos paquetes nos entran en el presupuesto?
        max_packages =  math.floor(self.max_budget / (asset.price*base_units))

        # Si no entra ni siquiera un paquete, no añadimos esta variable
        if max_packages < 1:
            continue

        # Tenemos que sumar 1 ya que si el número máximo es justo una potencia de dos
        # no cubriríamos todo (log2(8)=3, 2^0 + 2^1 +2^2 = 7)
        k = int(math.floor((math.log2(max_packages))+1))

        for i in range(k):
            package_multiplier = 2**i
            real_units = package_multiplier * base_units
            cost = real_units * asset.price
            val = cost * asset.roi
            self.variables.append(DecisionVariable(asset.name, real_units, cost, val, other_vars=self.variables))

@add_method(PortfolioProblem)
def create_slack_variables_packaged(self, min_package_pct=None ):
    """
    Estrategia: Variables de holgura (Slack), añadiendo el tamaño mínimo del paquete
    """
    if min_package_pct == None:
        self.create_slack_variables()
    else:
        min_money_package = self.max_budget * min_package_pct
        max_packages =  math.floor(self.max_budget / min_money_package)
        
        k = int(math.floor(math.log2(max_packages)+1))

        for i in range(k):
            quantity = 2**i
            val = quantity*min_money_package               
            self.variables.append(DecisionVariable("SLACK", int(quantity), float(val), 0.0, is_slack=True, other_vars=self.variables))   
        # Y una última variable con el relleno 
        holgura_final = self.max_budget - (max_packages * min_money_package)
        if holgura_final > 0:
            self.variables.append(DecisionVariable("SLACK", 0, float(holgura_final), 0.0, is_slack=True, other_vars=self.variables)) 


In [ ]:
# Definamos los parámetros de la primera aproximación, con un tamaño de paquete muy grande (15%)
min_package_pct = 0.15

problema_paquetes = problema_estatico.copy()
problema_paquetes.clear_variables()
problema_paquetes.max_budget = 4000
problema_paquetes.create_variables_binary_packaged(min_package_pct)
problema_paquetes.create_slack_variables_packaged(min_package_pct)
problema_paquetes.print_problem_definition(display_covariances = False, display_variables=True)

sol_info = solve_qubo_basico_risk(problema_paquetes)


### Limitar el peso de un activo en la cartera
Es habitual que no se quiera invertir más de una cierta cantidad en un único activo, con objeto de garantizar la diversificación. Esto es algo que el modelo implementará automáticamente al ser la varianza (riesgo intrínseco) normalmente mayor que la covarianza (riesgo extrínseco). Pero podemos forzar una restricción que nos ayude a limitar el número de variables. En lugar de tomar el tamaño de la cartera y crear variables para poder llenarla con cada activo financiero, tomaremos, por ejemplo, un 20% del tamaño de la cartera (si establecemos que nunca queremos que una inversión ocupe más del 20%). De este modo, serán necesarias menos variables adicionales.

In [ ]:
@add_method(PortfolioProblem)
def create_variables_with_max_weight(self, max_weight_per_asset, min_package_pct=0.05, fill_in_mode=0):
    """
    Genera variables binarias sin cubrir hasta el máximo del presupuesto, cubriendo solo
    hasta un máximo relativo por activo.

    fill_in_mode: 
        0: Exacto. La última variable se ajustará para cubir max_weight_per_asset
        1: Superar. La última variable permitirá superar max_weight_per_asset
        -1: Por debajo: La última variable completa quedará por debajo de max_weight_per_asset y no se añadirá nada más
    """
    min_money_package = self.max_budget * min_package_pct
    max_amt = max_weight_per_asset * self.max_budget
    if max_amt > self.max_budget:
        max_amt = self.max_budget    
    for asset in self.assets.values():
        # Cuantas acciones son un paquete?
        if asset.price >= min_money_package:
            base_units = 1
        else:
            base_units = round(min_money_package / asset.price)

        # Cuantos paquetes nos entran en el presupuesto?
        max_packages =  math.floor(max_amt / (asset.price*base_units))

        # Si no entra ni siquiera un paquete, no añadimos esta variable
        if max_packages < 1:
            continue

        # Tenemos que sumar 1 ya que si el número máximo es justo una potencia de dos
        # no cubriríamos todo (log2(8)=3, 2^0 + 2^1 +2^2 = 7)
        k = int(math.floor((math.log2(max_packages))+1))
        remaining_budget = max_amt
        for i in range(k):
            if remaining_budget < 0:
                break
            package_multiplier = 2**i
            real_units = package_multiplier * base_units
            cost = real_units * asset.price
            val = cost * asset.roi
            
            if remaining_budget < cost and fill_in_mode == 0:
                # Tenemos que corregir el limite. El último paquete tendrá solo lo que falta
                real_units = math.floor(remaining_budget/asset.price)
                cost = real_units * asset.price
                val = cost * asset.roi
            elif remaining_budget < cost and fill_in_mode == -1:
                # no añadimos este último paquete
                remaining_budget -= cost
                continue
            # Si hemos llegado aquí tenemos el último paquete corregido o completo según fill_in_mode sea 1 o -1
            self.variables.append(DecisionVariable(asset.name, real_units, cost, val, other_vars=self.variables))
            remaining_budget -= cost

In [ ]:
# Definamos los parámetros de la primera aproximación, con un tamaño de paquete muy grande (15%)
min_package_pct = 0.15
max_weight_per_asset = 0.40

problema_paquetes_limite = problema_estatico.copy()
problema_paquetes_limite.clear_variables()
problema_paquetes_limite.max_budget = 4000
problema_paquetes_limite.create_variables_with_max_weight(max_weight_per_asset,min_package_pct)
problema_paquetes_limite.create_slack_variables_packaged(min_package_pct)
problema_paquetes_limite.print_problem_definition(display_covariances = False, display_variables=True)

sol_info = solve_qubo_basico_risk(problema_paquetes_limite)


### Aproximación iterativa para refinar la solución
Podemos hacer una iteración inicial con paquetes de acciones relativamente grandes (por ejemplo el 5%-10% del tamaño máximo de la cartera) para obtener una solución aproximada. Luego, podemos redefinir el problema en torno a cada solución, estableciendo un mínimo y un máximo de inversión en ese activo:
- Mínimo: La cantidad invertida por la solución aproximada, menos un paquete completo del activo (con el tamaño de paquete de la ejecución anterior)
- Máximo: La cantidad invertida por la solución aproximada, más un paquete completo del activo (con el tamaño de paquete de la ejecución anterior)
Con dichos mínimos y máximos, podemos redefinir el problema con un tamaño de paquete menor y volver a obtener una solución más afinada.

Para ello, definiremos una variable inicial que representará un paquete de acciones para ese activo con el valor mínimo a invertir, y luego realizaremos la descomposición binaria del resto hasta el máximo a invertir. 

Esta alternativa nos permite obtener una solución aproximada y luego ir refinándola, para lograr soluciones a problemas con muchas variables, con recursos limitados. Como contraprestación, esta alternativa puede caer en mínimos locales: si la primera iteración da soluciones que no están en la zona del el mínimo global sino en la zona de otro mínimo, las siguientes iteraciones se acercarán a ese mínimo, pero no saltarán fácilmente al mínimo global.

In [ ]:
@add_method(PortfolioProblem)
def create_variables_with_bounds(self, bounds, min_package_pct=0.05, fill_in_mode=0):
    """
    Genera variables para cubrir un rango específico (coste de las acciones) [min, max] por activo.
    
    Para cada activo crea:
    - Una variable 'base' que cubre el mínimo solicitado.
    - Variables binarias (paquetes) para cubrir la diferencia (max - min).

    El parámetro bounds es un diccionario. Las claves son los nombres de los activos
    y los valores tuplas que contienen el mínimo y el máximo: name -> (min, max)


    fill_in_mode: 
        0: Exacto. La última variable se ajustará para cubir max_weight_per_asset
        1: Superar. La última variable permitirá superar max_weight_per_asset
        -1: Por debajo: La última variable completa quedará por debajo de max_weight_per_asset y no se añadirá nada más
    """
    min_money_package = self.max_budget * min_package_pct

    for name, asset in self.assets.items():
        if name not in bounds:
            continue #Si no está en la aproximación, no permitimos añadirla a la solución
        else:
            min_amt, max_amt = bounds[name]
        
        # Limitamos el máximo
        if max_amt > self.max_budget:
            max_amt = self.max_budget

        # Para la parte binaria, qué incrementos usamos?
        if asset.price >= min_money_package:
            base_units = 1
        else:
            base_units = round(min_money_package / asset.price)
            
        # Creamos la primera variable con el valor mínimo
        # Calculamos cuántas acciones caben en el importe mínimo
        units_at_min = math.floor(min_amt / asset.price)

        if units_at_min > 0:
            cost = units_at_min * asset.price
            val = cost * asset.roi
            # Esta variable representa el bloque "fijo" o mínimo esperado
            self.variables.append(DecisionVariable(asset.name, 1.0, cost, val, other_vars=self.variables))

        # Creamos las variables binarias hasta llegar al máximo
        # Calculamos el espacio que queda entre el máximo y lo que ya cubrimos con el mínimo
        units_at_max = math.floor(max_amt / asset.price)
        delta_units = units_at_max - units_at_min

        # Convertimos ese delta a número de "paquetes"
        packages_in_delta = delta_units // base_units

        if packages_in_delta < 1:
            # Si la diferencia es menor que un paquete base, no añadimos variables extra
            continue

        # Tenemos que sumar 1 ya que si el número máximo es justo una potencia de dos
        # no cubriríamos todo (log2(8)=3, 2^0 + 2^1 +2^2 = 7)
        k = int(math.floor(math.log2(packages_in_delta)+1))
        remaining_budget = max_amt
        for i in range(k):
            if remaining_budget < 0:
                break
            package_multiplier = 2**i
            real_units = package_multiplier * base_units
            cost = real_units * asset.price
            val = cost * asset.roi

            if remaining_budget < cost and fill_in_mode == 0:
                # Tenemos que corregir el limite. El último paquete tendrá solo lo que falta
                real_units = math.floor(remaining_budget/asset.price)
                cost = real_units * asset.price
                val = cost * asset.roi
            elif remaining_budget < cost and fill_in_mode == -1:
                # no añadimos este último paquete
                remaining_budget -= cost
                continue
            # Si hemos llegado aquí tenemos el último paquete corregido o completo según fill_in_mode sea 1 o -1
            self.variables.append(DecisionVariable(asset.name, real_units, cost, val, other_vars=self.variables))

In [ ]:
## Código para hacer el QUBO iterativo, y hacer varias iteraciones
import random
import string
import statistics



problema_iterativo = problema_estatico.copy()


# Definamos los parámetros de la primera aproximación, con un tamaño de paquete muy grande (15%)
min_package_pct = 0.15
max_weight_per_asset = 0.45

problema_iterativo.clear_variables()
problema_iterativo.max_budget = 4000
problema_iterativo.create_variables_with_max_weight(max_weight_per_asset,min_package_pct)
problema_iterativo.create_slack_variables_packaged(min_package_pct)
problema_iterativo.print_problem_definition(display_covariances = True, display_variables=True)


# Parametros fundamentales
# Hay que mantener el alpha y beta aunque ya no tengamos todas las variables, si no, cambian los resultados
sol_info = solve_qubo_basico_risk(problema_iterativo)




#Ahora podemos iterar para mejorar la solución a partir de la aproximación que tenemos

# Definamos los parámetros de la segunda aproximación, ahora con un paquete más pequeño (10% del anterior)
min_package_pct_orig = min_package_pct
min_package_pct = min_package_pct_orig * 0.1
max_weight_per_asset = 0.80

solution_table = problema_iterativo.get_solution_table(sol_info.solucion)

bounds = {}
for name, a in solution_table.items():
    min_amount = a['cost'] - min_package_pct_orig*problema_iterativo.max_budget
    max_amount = a['cost'] + min_package_pct_orig*problema_iterativo.max_budget
    bounds[name] = (min_amount, max_amount)

print ("\n\n\nBounds", bounds)

print ("Min package: ", min_package_pct)

problema_iterativo.clear_variables()
problema_iterativo.create_variables_with_bounds(bounds,min_package_pct)
problema_iterativo.create_slack_variables_packaged(min_package_pct)

problema_iterativo.print_problem_definition(display_covariances = False, display_variables=True)


solucion_qubo_iterativo = solve_qubo_basico_risk(problema_iterativo, sol_info.alpha, sol_info.beta)


